# Improvements

Author: Frankie Inguanez<br />
Date: 16/01/2023<br /><br />

Improvements on previous implementation.

In [1]:
import sudokuPuzzleUtils as spu
import sudokuGuessAlg as sga
import sudokuSearchAlg as ssa

In [2]:
## Adding tracking of solutions

class SudokuConfig:
    def __init__(self, searchMode: int, guessMode: int, tracking: bool):
        self.searchMode=searchMode
        self.guessMode=guessMode
        self.tracking=tracking

In [3]:
def backtracking(board: list, history: list, stats: spu.SudokuStats, config: SudokuConfig):
    """
    Solves a 9x9 sudoku puzzle using backtracking algorithm.
    Arguments:
        board: the 9x9 puzzle to be solved.
        stats: The statistics object to record algorithm.
        searchMode: defines how the puzzle is parsed: 1 by row; 2 by col; 3 by box sequentially; 4 by box in a zig-zag; 5 by box in a spiral; 6 by box in a semi-zig-zag
        guessMode: defines how numbers are guessed: 1 sequentially; 2 randomly
    """
    # Find the next empty cell
    find = ssa.findEmpty(board, config.searchMode)

    # If there is no empty cell than puzzle is complete
    if not find:
        return True
    else:
        row, col = find

    # Get numbers to guess and attempt
    for guess in sga.getGuesses(board, config.guessMode):
        if spu.isValid(board, guess, (row, col)):

            # Brute force guess
            stats.incrementGuesses()
            board[row][col] = guess

            if config.tracking:
                history.append(spu.toStr(board))

            # Attempt to solve rest of puzzle with current choice
            if backtracking(board, history, stats, config):
                return True

            # Invalid puzzle so backtrack
            if (config.tracking):
                history.remove(spu.toStr(board))
            stats.incrementBacktracks()
            board[row][col] = 0

    return False

In [4]:
board = spu.to2DArray('100008509000000000460000000600000000085020047094600302000003090920000700000107005')
board

[[1, 0, 0, 0, 0, 8, 5, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [4, 6, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 8, 5, 0, 2, 0, 0, 4, 7],
 [0, 9, 4, 6, 0, 0, 3, 0, 2],
 [0, 0, 0, 0, 0, 3, 0, 9, 0],
 [9, 2, 0, 0, 0, 0, 7, 0, 0],
 [0, 0, 0, 1, 0, 7, 0, 0, 5]]

In [5]:
import timeit

history = []
stats = spu.SudokuStats()
config = SudokuConfig(searchMode=1, guessMode=1, tracking=True)

timeit.timeit(lambda:backtracking(board, history, stats, config), number=1000)

14.403690400002233

In [6]:
board

[[1, 3, 7, 4, 6, 8, 5, 2, 9],
 [2, 5, 8, 3, 1, 9, 4, 7, 6],
 [4, 6, 9, 5, 7, 2, 1, 8, 3],
 [6, 1, 2, 7, 3, 4, 9, 5, 8],
 [3, 8, 5, 9, 2, 1, 6, 4, 7],
 [7, 9, 4, 6, 8, 5, 3, 1, 2],
 [5, 7, 6, 2, 4, 3, 8, 9, 1],
 [9, 2, 1, 8, 5, 6, 7, 3, 4],
 [8, 4, 3, 1, 9, 7, 2, 6, 5]]

In [7]:
history

['130008509000000000460000000600000000085020047094600302000003090920000700000107005',
 '137008509000000000460000000600000000085020047094600302000003090920000700000107005',
 '137408509000000000460000000600000000085020047094600302000003090920000700000107005',
 '137468509000000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529000000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529200000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529250000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529258000000460000000600000000085020047094600302000003090920000700000107005',
 '137468529258300000460000000600000000085020047094600302000003090920000700000107005',
 '137468529258310000460000000600000000085020047094600302000003090920000700000107005',
 '137468529258319000460000000600000000085020047094600302000003090920000700000107005',
 '1374685292583194004600000006000000000850200470946003

In [8]:
for i in range(len(history)):
    print(history[i])
    break

130008509000000000460000000600000000085020047094600302000003090920000700000107005


In [9]:
def generatePuzzle(count: int, zeros: int):
    """
    Generates a 9x9 sudoku puzzle grid.
    Arguments:
        count: the number of puzzles to generate.
        zeros: the number of zeros to inject in the puzzles.
    """
    import random
    
    puzzles = []
    for i in range(count):
        puzzle = '000000000000000000000000000000000000000000000000000000000000000000000000000000000'

        board = spu.to2DArray(puzzle)
        stats = spu.SudokuStats()
        stats.setUnknowns(puzzle.count('0'))
        config = SudokuConfig(searchMode=9, guessMode=2, tracking=False)
        backtracking(board, None, stats, config)

        # Remove digits
        changed = 0
        while changed < zeros:
            row = random.sample(range(0,9),1)[0]
            col = random.sample(range(0,9),1)[0]
            
            if board[row][col]==0:
                continue

            board[row][col]=0
            changed+=1
            
        puzzles.append(spu.toStr(board))

    return puzzles
    

In [10]:
generatePuzzle(2,54)

['001000480000000001006400000040300000018004300520008704967140800000560003000000900',
 '500687020900302400280000005000100002000800700000020004008200007000930006006700800']

In [12]:
import tqdm

# Generate balanced dataset of 50 puzzles for different number of missing digits
with open("balanced_24_45_zeros.txt", "w", encoding="utf-8") as f:
    for i in tqdm.tqdm(range(24,46)):
        puzzles = generatePuzzle(50,i)

        for j in range(len(puzzles)):
            f.write("{}\n".format(spu.toStr(puzzles[j])))


100%|██████████| 22/22 [07:26<00:00, 20.29s/it]
